## <span style=color:blue>This notebook is fetching soybean yields for an ML pipeline </spann>

<span style=color:blue>It pulls from USDA NASS.</span>


In [44]:
# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

### <span style=color:blue> Accessing USDA NASS, following code from https://towardsdatascience.com/harvest-and-analyze-agricultural-data-with-the-usda-nass-api-python-and-tableau-a6af374b8138.  In first cell below we define a class for interacting with the NASS QuickStats API, and in second cell we illustrate how to invoke that class </span>

In [45]:
# from https://towardsdatascience.com/harvest-and-analyze-agricultural-data-with-the-usda-nass-api-python-and-tableau-a6af374b8138
# with edits

#   Name:           c_usda_quick_stats.py
#   Author:         Randy Runtsch
#   Date:           March 29, 2022
#   Project:        Query USDA QuickStats API
#   Author:         Randall P. Runtsch
#
#   Description:    Query the USDA QuickStats api_GET API with a specified set of 
#                   parameters. Write the retrieved data, in CSV format, to a file.
#
#   See Quick Stats (NASS) API user guide:  https://quickstats.nass.usda.gov/api
#   Request a QuickStats API key here:      https://quickstats.nass.usda.gov/api#param_define
#
#   Attribution: This product uses the NASS API but is not endorsed or certified by NASS.
#
#   Changes
#

import urllib.request
from urllib.error import HTTPError
from requests.utils import requote_uri
import requests

# Retrieve NASS API key from environment variables (you have to get your own)
import os
my_NASS_API_key = os.getenv('NASS_API_KEY')

class c_usda_quick_stats:

    def __init__(self):

        # Set the USDA QuickStats API key, API base URL, and output file path where CSV files will be written. 

        # self.api_key = 'PASTE_YOUR_API_KEY_HERE'
        self.api_key = "62A21C68-28D5-3C34-B09F-1E68D569ABE3"

        self.base_url_api_get = 'http://quickstats.nass.usda.gov/api/api_GET/?key=' \
                                + self.api_key + '&'

    def get_data(self, parameters, file_path, file_name):

        # Call the api_GET api with the specified parameters. 
        # Write the CSV data to the specified output file.

        # Create the full URL and retrieve the data from the Quick Stats server.
        
        full_url = self.base_url_api_get + parameters        
        print(full_url)

        try:
            s_result = urllib.request.urlopen(full_url)
            # print(type(s_result))
            print(s_result.status, s_result.reason)
            # print(s_result.status_code)
            s_text = s_result.read().decode('utf-8')

            # Create the output file and write the CSV data records to the file.

            s_file_name = file_path + file_name
            o_file = open(s_file_name, "w", encoding="utf8")
            o_file.write(s_text)
            o_file.close()
        except HTTPError as error:
            print(error.code, error.reason)
        except requests.exceptions.RequestException as e:
            print(f"An error occurred while fetching the data: {e}")
        except ValueError as e:
            print(f"Failed to parse the response data: {e}")
        except:
            print(f"Failed because of unknown exception; perhaps the USDA NASS site is down")


<span style=color:blue>Now a query that fetches useful soybean yield data.  I am focused on the top 7 soy-producing states in the US, and on the years 2003 to 2022.   </span>

In [47]:
import sys
import urllib.parse

output_dir = './yield_data/'

# Create a string with search parameters, then create an instance of
# the c_usda_quick_stats class and use that to fetch data from QuickStats
# and write it to a file

# It took a while to get the parameter names just right...
#   The parameters names are listed in
#      https://quickstats.nass.usda.gov/param_define
#   (some additional resources in https://quickstats.nass.usda.gov/tutorials)
#   Also, look at the column names that show up in the csv files that you get back
parameters =    'source_desc=SURVEY' +  \
                '&sector_desc=CROPS' + \
                '&' + urllib.parse.quote('group_desc=FIELD CROPS') + \
                '&commodity_desc=Wheat' + \
                '&statisticcat_desc=YIELD' + \
                '&geographic_level=STATE' + \
                '&agg_level_desc=COUNTY' + \
                '&state_ansi=53' + \
                '&state_ansi=30' + \
                '&state_ansi=38' + \
                '&state_ansi=46' + \
                '&state_ansi=20' + \
                '&state_ansi=40' + \
                '&state_ansi=48' + \
                '&year__GE=2000' + \
                '&year__LE=2023' + \
                '&format=CSV'

stats = c_usda_quick_stats()

# holding this timestamp; we may used it to import the created csv file
latest_curr_timestamp = curr_timestamp()
filename = 'wheat_yield_data__' + latest_curr_timestamp + '.csv'

# Including curr_timestamp() into file name to keep outputs separated during development/exploration
stats.get_data(parameters, output_dir, filename)



http://quickstats.nass.usda.gov/api/api_GET/?key=62A21C68-28D5-3C34-B09F-1E68D569ABE3&source_desc=SURVEY&sector_desc=CROPS&group_desc%3DFIELD%20CROPS&commodity_desc=Wheat&statisticcat_desc=YIELD&geographic_level=STATE&agg_level_desc=COUNTY&state_ansi=53&state_ansi=30&state_ansi=38&state_ansi=46&state_ansi=20&state_ansi=40&state_ansi=48&year__GE=2000&year__LE=2023&format=CSV
200 OK


### <span style=color:blue>After inspecting the output we see that there is double counting.  In particular, see the columns for "short_desc".  So, we will drop all records with short_desc != "SOYBEANS - YIELD, MEASURED IN BU / ACRE"</span>

In [48]:
import pandas as pd

output_dir = './yield_data/'

df = pd.read_csv(output_dir + filename)
# print(df.head())

df1 = df[['short_desc']].drop_duplicates()
print(df1.head(10))
print()

# keep only records about full yield
df = df[df['short_desc'] == "WHEAT - YIELD, MEASURED IN BU / ACRE"]
print(len(df))
# 10295

print()

# found some bad_county_names by visual inspection of the csv
bad_county_names = ['OTHER COUNTIES', 'OTHER (COMBINED) COUNTIES']
df = df[~df.county_name.isin(bad_county_names)]

print(len(df))
# 9952

print()

df2 = df[['state_name','county_name']].drop_duplicates()
print(len(df2))
# 559

# Note: using SQL I found that of the 559 state-county pairs total:
#          212 state-county pairs have data for all 20 years
#          347 state-county pairs have data for < 20 years
#
#          486 have year 2022
#          418 have year 2021
#          514 have year 2020
# I will live with that

# cleaning up a column name
df = df.rename(columns={'Value': 'yield'})

output_dir = './yield_data/'
output_file = 'repaired_yield__' + curr_timestamp() + '.csv'

df.to_csv(output_dir + output_file, index=False)

# I imported this table into postgres so that I could use SQL ...

                                              short_desc
0                   WHEAT - YIELD, MEASURED IN BU / ACRE
4189    WHEAT - YIELD, MEASURED IN BU / NET PLANTED ACRE
5466        WHEAT - YIELD, MEASURED IN BU / PLANTED ACRE
5601     WHEAT, IRRIGATED - YIELD, MEASURED IN BU / ACRE
7011   WHEAT, IRRIGATED - YIELD, MEASURED IN BU / NET...
7425   WHEAT, IRRIGATED - YIELD, MEASURED IN BU / PLA...
7447   WHEAT, NON-IRRIGATED - YIELD, MEASURED IN BU /...
10525  WHEAT, NON-IRRIGATED - YIELD, MEASURED IN BU /...
11641  WHEAT, NON-IRRIGATED - YIELD, MEASURED IN BU /...
11718  WHEAT, NON-IRRIGATED, CONTINUOUS CROP - YIELD,...

4189

3988

557


#### <span style=color:blue>Saving the csv I'm happy with in a designated place in my "archives" directory</span>

In [49]:
import shutil

output_dir = './yield_data/'
archives_dir = './yield_data/'
src_file = output_file # from preceding cell
tgt_file = 'wheat_yield_data.csv'

shutil.copyfile(output_dir + src_file, archives_dir + tgt_file)

'./yield data/wheat_yield_data.csv'

#### <span style=color:blue>Projecting out the columns and records that I don't need for my ML learning table, and archiving that result, also. </span>

In [50]:
import pandas as pd

archives_dir = './yield_data/'
tgt_file = 'wheat_yield_data.csv'

df = pd.read_csv(archives_dir + tgt_file)
# print(df.head())

cols_to_keep = ['year','state_name','county_name','yield']
dfml = df[cols_to_keep]

print(dfml.head())
print()
print(dfml.shape[0])
# Note: this particular df has 9952 rows

# checking there are no null values for 'yield':
print(dfml[dfml['yield'].isnull()].head())

tgt_file_01 = 'd'
dfml.to_csv(archives_dir + tgt_file_01, index=False)
print('\nwrote file ', archives_dir + tgt_file_01)

   year state_name county_name  yield
0  2007     KANSAS    CHEYENNE   47.0
1  2006     KANSAS    CHEYENNE   20.0
2  2005     KANSAS    CHEYENNE   21.0
3  2004     KANSAS    CHEYENNE   18.0
4  2003     KANSAS    CHEYENNE   32.0

3988
Empty DataFrame
Columns: [year, state_name, county_name, yield]
Index: []

wrote file  ./yield data/d
